# Introduction   
I love espresso, particularly that of local places like Intelligentsia, Bowtruss, and Metropolis on weekdays for breaks. I currently don't work too close to any of these places so perhaps a small espresso bar, or stand would be a good idea to consider. I like looking at data geospatially and would like to the same with this: 
- At what stops is there less competition for a little espresso bar/booth and would the stop nearest to my office be one? 
- Which would be the least potentially good, and does that line up with my regular coffee consumption habits?
- What is coffee consumption for a given neighborhood?  

***For full disclosure, I've had the ridership and station information datasets on my computer for some time, and have already done some exploratory analyses of them. The other dataset I use below, of business licences I used for a a challenge in the past.***

## Procedure  
### 1) Examine given data  
Given the starting ridership data the first obvious filter to apply is for daytype, since we're only interested in weekday data, let's filter for just that. Also, to reduce process time I limited the date to start in 2011. 

In [1]:
# set up environment
import pandas as pd
from bokeh.charts import show,output_notebook,Bar,Histogram

output_notebook()

Loading BokehJS ...

In [2]:
# import ridership data
cta = pd.read_table('CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals.tsv',parse_dates=[2])
cta.shape

(809326, 5)

In [3]:
# filter data
cta_after_2011 = cta[cta['date']>='01/01/2011']
weekdayTrain = cta_after_2011[cta_after_2011['daytype']=='W'].reset_index(drop=True)

# transform ridership data
avgStationVolume = pd.DataFrame(weekdayTrain.pivot_table(index=['station_id','stationname'],values='rides')).reset_index()
avgStationVolume.shape

(145, 3)

### 2) Add in station location data  
The data to start isn't the most to work with and because my question is geospatially-focused it would be good to add the cta_station info data sate from the Chicago data portal too. 

In [4]:
# import station data 
stationInfo = pd.read_table('cta_stations.tsv',usecols=['MAP_ID','Location'])

# a little cleanup and manipulation
stationInfo = stationInfo.drop_duplicates().reset_index(drop=True)
stationInfo.columns = ['station_id','location']

# parse coordinates
stationInfo['lat'] = stationInfo['location'].apply(lambda x: x[1:-1].split(', ')[0])
stationInfo['long'] = stationInfo['location'].apply(lambda x: x[1:-1].split(', ')[1])

allStationData = avgStationVolume.merge(stationInfo,on='station_id')[[0,1,2,4,5]]
print(allStationData.shape)
allStationData.head()

(146, 5)


,station_id,stationname,rides,lat,long
0,40010,Austin-Forest Park,2092.652875,41.870851,-87.776812
1,40020,Harlem-Lake,3903.284912,41.886848,-87.803176
2,40030,Pulaski-Lake,1890.245971,41.885412,-87.725404
3,40040,Quincy/Wells,8033.538838,41.878723,-87.63374
4,40050,Davis,3841.691011,42.04771,-87.683543


### 3) Business data
Now to integrate the information on those business I mentioned in the introduction. Luckily, I'm familar with the Business Licence dataset on the Chicago Data portal and remembered that it has multiple names, in addition to geospatial information already baked in. Below is not the __cleanest__ processing but to get a quick and dirty hold on the data it will do. I pared down the data to businesses with five or more unique locations for a legal name, so some places that I regularly visit, like The Wormhole Coffee off the Damen Blue Line stop or Sawada Coffee in West Loop, don't end up being evaluated.

In [5]:
# import business data
businessData = pd.read_table('Coffee_Tea.tsv')

# pare down the data so it's easier to browse
businessData = businessData[['DOING BUSINESS AS NAME','LEGAL NAME','LATITUDE','LONGITUDE']]
businessData = businessData.drop_duplicates().dropna().reset_index(drop=True)
businessData = businessData.sort_values('DOING BUSINESS AS NAME')

# reduce the data down to organizations that have 5 or more locations (arbitrary number)
pt = pd.DataFrame(businessData.pivot_table(index='LEGAL NAME',values='DOING BUSINESS AS NAME',aggfunc='count'))
reducedSet = pt[pt['DOING BUSINESS AS NAME']>=5].reset_index()[[0]]

# append a binary rating to the filtered data
eugeneGoodCoffee = ['BOW & TRUSS, LLC','INTELLIGENTSIA COFFEE & TEA INC.','METROPOLIS COFFEE COMPANY LLC']
reducedSet['espresso'] = reducedSet['LEGAL NAME'].apply(lambda x: 1 if x in eugeneGoodCoffee else 0)

# filter and merge 
reducedBusinessData = businessData[businessData['LEGAL NAME'].isin(reducedSet['LEGAL NAME'])].reset_index(drop=True)
reducedBusinessData = reducedBusinessData.merge(reducedSet)
reducedBusinessData = reducedBusinessData.drop_duplicates()
reducedBusinessData.head()

,DOING BUSINESS AS NAME,LEGAL NAME,LATITUDE,LONGITUDE,espresso
0,ACE COFFEE BAR,"ACE COFFEE BAR, INC.",41.842996,-87.695309,0
1,ACE COFFEE BAR INC,"ACE COFFEE BAR, INC.",41.941426,-87.690799,0
2,ACE COFFEE BAR INC,"ACE COFFEE BAR, INC.",41.866906,-87.673316,0
3,ACE COFFEE BAR INC,"ACE COFFEE BAR, INC.",41.863501,-87.642210,0
4,ACE COFFEE BAR INC.,"ACE COFFEE BAR, INC.",41.803731,-87.622545,0


In [6]:
# Export for QGIS
# allStationData.to_csv('stationData.csv')
# reducedBusinessData.to_csv('businessData.csv')

### 4) Geospatial Processing with QGIS  
I'm a big fan of freeware, especially QGIS, which one of my friends calls malware with geospatial function. Handily I remember how to do some basic vector and clustering procedures then visualize them (nice for quick and dirty geospatial things). This processing yielded a dataset of all of the stations and their five nearest businesses.  
  
#### Here's a screen short of the station and business data rendered in QGIS  
![this](img\spatial_start.jpg)
  
#### And a bit of nearest neighbor in QGIS

![this one too](img\cluster_analysis.jpg)

### 5) Answering some questions
Now that we've established mapping from train stops to businesses we can get a better idea of viability for our little espresso business.

In [7]:
# load the QGIS distance matrix and get an idea of what it looks like
stationClusters = pd.read_csv('distance_matrix.csv')
stationClusters.columns = ['station_idx','business_idx','distance']
stationClusters.head()

,station_idx,business_idx,distance
0,0,242,0.046093
1,0,271,0.059941
2,0,259,0.059941
3,0,278,0.060990
4,0,243,0.060990


In [8]:
# prep station data for merge
resetStationData = allStationData.reset_index()
resetStationData.columns = ['station_idx', 'station_id', 'stationname', 'rides', 'lat', 'long']

# prep business data for merge
resetBusinessData = reducedBusinessData.reset_index()
resetBusinessData.columns = ['business_idx', 'DOING BUSINESS AS NAME', 'LEGAL NAME', 'LATITUDE','LONGITUDE', 'espresso']

# merge both prepped datasets
mergedData = stationClusters.merge(resetStationData).merge(resetBusinessData)
mergedData = mergedData[['station_idx', 'business_idx', 'station_id', 'stationname',
       'rides', 'DOING BUSINESS AS NAME', 'LEGAL NAME','espresso']]

# We can now see which stations have the best espresso scores
espresso = mergedData.pivot_table(values='espresso',columns=['stationname','station_idx','station_id']).sort_values(ascending=False).reset_index()
espresso.columns = ['stationname', 'station_idx', 'station_id', 'score']
espresso.head(10)

,stationname,station_idx,station_id,score
0,Damen/Milwaukee,56,40590,0.6
1,Division/Milwaukee,31,40320,0.6
2,Logan Square,96,41020,0.4
3,Cicero-Lake,46,40480,0.4
4,Belmont-O'Hare,5,40060,0.4
5,Lake/State,142,41660,0.4
6,State/Lake,25,40260,0.4
7,Randolph/Wabash,19,40200,0.4
8,Merchandise Mart,44,40460,0.2
9,LaSalle/Van Buren,15,40160,0.2


## Results

Let's take a look at some of resulting data, evaluating espresso scores overall, and then by examining some specific train stops. 

In [9]:
p = Histogram(espresso[[3]],legend=False,title='Espresso Scores by Count')
p.toolbar_location=None
show(p)

In [10]:
# CTA stop closest to my current stop
mergedData[mergedData['station_id']==41160]

,station_idx,business_idx,station_id,stationname,rides,DOING BUSINESS AS NAME,LEGAL NAME,espresso
562,108,255,41160,Clinton-Lake,4290.380183,Starbucks Coffee #23421,STARBUCKS CORPORATION,0
568,108,144,41160,Clinton-Lake,4290.380183,STARBUCKS COFFEE #11048,STARBUCKS CORPORATION,0
703,108,14,41160,Clinton-Lake,4290.380183,Beavers Coffee & Donuts,"AFTER HOURS PIZZA, LLC",0
704,108,7,41160,Clinton-Lake,4290.380183,BEAVERS COFFEE & DONUTS,"AFTER HOURS PIZZA, LLC",0
705,108,238,41160,Clinton-Lake,4290.380183,STARBUCKS COFFEE #2877,STARBUCKS CORPORATION,0


In [11]:
# Place with the best espresso score
mergedData[mergedData['station_idx']==56]

,station_idx,business_idx,station_id,stationname,rides,DOING BUSINESS AS NAME,LEGAL NAME,espresso
473,56,95,40590,Damen/Milwaukee,6279.384454,Intelligentsia Coffee & Tea,INTELLIGENTSIA COFFEE & TEA INC.,1
475,56,26,40590,Damen/Milwaukee,6279.384454,Bow Truss Coffee Roasters,"BOW & TRUSS, LLC",1
609,56,214,40590,Damen/Milwaukee,6279.384454,STARBUCKS COFFEE #2443,STARBUCKS CORPORATION,0
611,56,97,40590,Damen/Milwaukee,6279.384454,Intelligentsia Coffee & Tea,INTELLIGENTSIA COFFEE & TEA INC.,1
612,56,153,40590,Damen/Milwaukee,6279.384454,STARBUCKS COFFEE #14256,STARBUCKS CORPORATION,0


In [12]:
# For fun, what is it like around my closest train stop?
mergedData[mergedData['station_id']==40880]

,station_idx,business_idx,station_id,stationname,rides,DOING BUSINESS AS NAME,LEGAL NAME,espresso
485,83,64,40880,Thorndale,2985.707368,DUNKIN DONUT,DUNKIN DONUTS,0
490,83,215,40880,Thorndale,2985.707368,STARBUCKS COFFEE #2445,STARBUCKS CORPORATION,0
651,83,109,40880,Thorndale,2985.707368,METROPOLIS COFFEE COMPANY,METROPOLIS COFFEE COMPANY LLC,1
654,83,57,40880,Thorndale,2985.707368,"Dinky Donuts, Inc.","DINKY DONUTS, INC.",0
658,83,128,40880,Thorndale,2985.707368,STABUCKS COFFEE #19283,STARBUCKS CORPORATION,0


## Conclusion

It's not clear whether my espresso cart idea would work near my current workplace, with only average rides and espresso scores for larger chains to to use, but given the espresso scores for, the Damen/Milwaukee stop would certainly not be great- too much competition. But hey, it is also the stop closest to one of my favorite spots, The Wormhole, so that reinforces confidence in my current favorite stop for coffee. 

Most stops had no, if any espresso score so it's not clear whether they'd make a viable location to start my little business near them. 

### Improvements
First I would add more metrics, like coffee per capita in Chicago, popularity of different spots, using data from Yelp or FourSquare. Some of these datasets, the Yelp and FourSquare datasets in particular would be good for discovering new places that could give an espresso score of 1. It would be interesting to see if other indicators of 'good coffee' would arise.   
  
  I also ran into some issues with QGIS and layer renderings, so I found it difficult to extract specific neighborhoods certain scores fell into. I know R's geospatial packages are rather intuitive and easy to implement that may be a better venue to implement more vector analysis. 
  
### Closing Comments
This was a cool prompt to get my feet more wet with these civic data sets, though some have been sitting around on my computer for a while. Thanks for your time and consideration and I look forward to your response. Perhaps I'll find a cool new place to get a nice coffee too.  
  
  Thanks,  
      B. Eugene Smith